In [5]:
import pandas as pd
import numpy as np

_feed = '/opt/app/data/data/TopofBook20130206/TopofBook20130206.txt'
%time data = pd.read_csv(_feed, low_memory=False)

CPU times: user 4.12 s, sys: 1.1 s, total: 5.22 s
Wall time: 5.6 s


In [4]:
data.shape

(6328950, 6)

In [8]:
data.dtypes

SECCODE     object
BUYSELL     object
TIME         int64
TRADENO    float64
PRICE       object
VOLUME      object
dtype: object

In [9]:
data.SECCODE.unique()

array(['USD000UTSTOM', 'BKTRUB000TOM', 'USD000000TOD', 'EUR_RUB__TOD',
       'EUR_RUB__TOM', 'EUR000TODTOM', 'USD000TODTOM', 'EURUSDTODTOM',
       'EURUSD000TOM', 'EURUSD000TOD', 'CNY000000TOD', 'USDRUB_TOM1M',
       'USDRUB_TOM1W', 'USDRUB_TOM1Y', 'USDRUB_TOM1D', 'USDRUB_TOM2M',
       'USDRUB_TOM2W', 'USDRUB_TOM3M', 'USDRUB_TOM6M', 'USDRUB_TOM9M',
       'BYR_RUB__TOM'], dtype=object)

In [20]:
%time data.SECCODE.value_counts()

CPU times: user 1.01 s, sys: 10 ms, total: 1.02 s
Wall time: 1.01 s


EUR_RUB__TOM    2405788
BKTRUB000TOM    1549432
USD000UTSTOM    1513218
EUR_RUB__TOD     536156
USD000000TOD     159622
EURUSD000TOM     117919
EURUSD000TOD      38133
USD000TODTOM       3071
EUR000TODTOM       2006
USDRUB_TOM6M        958
USDRUB_TOM1Y        754
CNY000000TOD        529
USDRUB_TOM9M        477
USDRUB_TOM3M        316
USDRUB_TOM1M        316
EURUSDTODTOM        174
USDRUB_TOM1W         26
USDRUB_TOM1D         19
USDRUB_TOM2W         17
USDRUB_TOM2M         17
BYR_RUB__TOM          2
Name: SECCODE, dtype: int64

In [25]:
%time data[data.SECCODE=='EURUSD000TOD'].describe().T

CPU times: user 590 ms, sys: 20 ms, total: 610 ms
Wall time: 596 ms


,count,mean,std,min,25%,50%,75%,max
TIME,38133.0,1.242897e+08,1.229879e+07,100003656.0,1.139024e+08,122913292.0,133440227.0,145959884.0
TRADENO,22.0,7.614983e+07,6.636021e+03,76140508.0,7.614250e+07,76150438.0,76156058.0,76157753.0
